In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import os
import plotly.graph_objects as go
from kernels import IonRate
from field_functions import LaserField
from scipy.integrate import simpson

from plotting import plotter

class AU:
    meter = 5.2917720859e-11 # atomic unit of length in meters
    nm = 5.2917721e-2 # atomic unit of length in nanometres
    second = 2.418884328e-17 # atomic unit of time in seconds
    fs = 2.418884328e-2 # atomic unit of time in femtoseconds
    Joule = 4.359743935e-18 # atomic unit of energy in Joules
    eV = 27.21138383 # atomic unit of energy in electronvolts
    Volts_per_meter = 5.142206313e+11 # atomic unit of electric field in V/m
    Volts_per_Angstrom = 51.42206313 # atomic unit of electric field in V/Angström
    speed_of_light = 137.035999 # vacuum speed of light in atomic units
    Coulomb = 1.60217646e-19 # atomic unit of electric charge in Coulombs
    PW_per_cm2_au = 0.02849451308 # PW/cm^2 in atomic units
AtomicUnits=AU

params = {
    'E_g': 0.5, 
    'αPol': 4.51, 
    "div_p":2**-4*32, 
    "div_theta":1*8, 
    'lam0': 450, 
    'intensity': 8e13, 
    'cep': 0, 
    'excitedStates': 1, 
    'coeffType': 'trecx', 
    'gauge': 'length', 
    'get_p_only': True, 
    'only_c0_is_1_rest_normal': False, 
    'abs_normal_phase_0': False,
    'delay': None, 
    'plotting': True
}

# to save computation time we can neglect cross terms!! just look at formula for <p|d|psi> there is i^l and because of the transition rules l has to be +-1 so if we sum over all states and one is complex conjugatet and we sum it up they will cancel out !!! but be carefull we only can use states that are allowed theoretically
# excitedStates = 0 should rduce only ground state with stark effect => if excitedStates != None

In [2]:
%autoreload 2
laser_pulses = LaserField(cache_results=True)
laser_pulses.add_pulse(params['lam0'], params['intensity'], params['cep'], params['lam0']/ AtomicUnits.nm / AtomicUnits.speed_of_light)
t_min, t_max = laser_pulses.get_time_interval()
time_recon= np.arange(int(t_min), int(t_max)+1, 0.25)

rate_SFA = IonRate(time_recon, laser_pulses, params, dT=0.5/8, kernel_type='exact_SFA')
laser_pulses.reset()

In [3]:
%autoreload 2
laser_pulses.add_pulse(params['lam0'], params['intensity'], params['cep'], params['lam0']/ AtomicUnits.nm / AtomicUnits.speed_of_light)
t_min, t_max = laser_pulses.get_time_interval()
time_recon_1= np.arange(int(t_min), int(t_max)+1, 0.25)
params['coeffType'] = 'trecx'
params['only_c0_is_1_rest_normal'] = True

rateExcited_1 = IonRate(time_recon_1, laser_pulses, params, dT=0.5/8, kernel_type='exact_SFA', excitedStates=True)
laser_pulses.reset()

0 0
only_c0_is_1_rest_normal is set to True


In [4]:
%autoreload 2
laser_pulses.add_pulse(params['lam0'], params['intensity'], params['cep'], params['lam0']/ AtomicUnits.nm / AtomicUnits.speed_of_light)
t_min, t_max = laser_pulses.get_time_interval()
time_recon_2= np.arange(int(t_min), int(t_max)+1, 0.25)
params['coeffType'] = 'trecx'
params['only_c0_is_1_rest_normal'] = False
params['abs_normal_phase_0'] = True

rateExcited_2 = IonRate(time_recon_2, laser_pulses, params, dT=0.5/8, kernel_type='exact_SFA', excitedStates=True)
laser_pulses.reset()

0 0
abs_normal_phase_0 is set to True


In [5]:
%autoreload 2
laser_pulses.add_pulse(params['lam0'], params['intensity'], params['cep'], params['lam0']/ AtomicUnits.nm / AtomicUnits.speed_of_light)
t_min, t_max = laser_pulses.get_time_interval()
time_recon_3 = np.arange(int(t_min), int(t_max)+1, 0.25)
params['coeffType'] = 'numerical'
params['only_c0_is_1_rest_normal'] = True
params['abs_normal_phase_0'] = False

rateExcited_3 = IonRate(time_recon_3, laser_pulses, params, dT=0.5/8, kernel_type='exact_SFA', excitedStates=True)
laser_pulses.reset()

Basis states (6): [(1, 0, 0), (2, 0, 0), (2, 1, 0), (3, 0, 0), (3, 1, 0), (3, 2, 0)]
0 0
only_c0_is_1_rest_normal is set to True


In [6]:
%autoreload 2
laser_pulses.add_pulse(params['lam0'], params['intensity'], params['cep'], params['lam0']/ AtomicUnits.nm / AtomicUnits.speed_of_light)
t_min, t_max = laser_pulses.get_time_interval()
time_recon_4 = np.arange(int(t_min), int(t_max)+1, 0.25)
params['coeffType'] = 'numerical'
params['only_c0_is_1_rest_normal'] = False
params['abs_normal_phase_0'] = True

rateExcited_4 = IonRate(time_recon_4, laser_pulses, params, dT=0.5/8, kernel_type='exact_SFA', excitedStates=True)
laser_pulses.reset()

Basis states (6): [(1, 0, 0), (2, 0, 0), (2, 1, 0), (3, 0, 0), (3, 1, 0), (3, 2, 0)]
0 0
abs_normal_phase_0 is set to True


In [7]:
import pandas as pd

data_dict = {
    'time_SFA': time_recon,
    'rate_SFA': rate_SFA,
    'time_extended_1': time_recon_1,
    'rate_extended_1': np.real(rateExcited_1),
    'time_extended_2': time_recon_2, 
    'rate_extended_2': np.real(rateExcited_2),
    'time_extended_3': time_recon_3,
    'rate_extended_3': np.real(rateExcited_3),
    'time_extended_4': time_recon_4,
    'rate_extended_4': np.real(rateExcited_4)
}

df = pd.DataFrame(data_dict)

filename = f"ionization_rates_{params['lam0']}nm_{params['intensity']:.1e}Wcm2.csv"
df.to_csv(filename, index=False)

print(f"Data exported to: {filename}")
print(f"DataFrame shape: {df.shape}")
print("\nFirst few rows:")
print(df.head())

summary_dict = {
    'Method': ['Standard SFA', 'Extended SFA (trecx, |c0|=1)', 'Extended SFA (trecx, phase only)', 
               'Extended SFA (numerical, |c0|=1)', 'Extended SFA (numerical, phase only)'],
    'Integrated_Yield': [
        np.trapz(rate_SFA, time_recon),
        np.real(np.trapz(rateExcited_1, time_recon_1)),
        np.real(np.trapz(rateExcited_2, time_recon_2)),
        np.real(np.trapz(rateExcited_3, time_recon_3)),
        np.real(np.trapz(rateExcited_4, time_recon_4))
    ],
    'Parameters': [
        'Standard',
        'coeffType=trecx, only_c0_is_1_rest_normal=True',
        'coeffType=trecx, abs_normal_phase_0=True',
        'coeffType=numerical, only_c0_is_1_rest_normal=True', 
        'coeffType=numerical, abs_normal_phase_0=True'
    ]
}

summary_df = pd.DataFrame(summary_dict)
summary_filename = f"ionization_summary_{params['lam0']}nm_{params['intensity']:.1e}Wcm2.csv"
summary_df.to_csv(summary_filename, index=False)

print(f"\nSummary exported to: {summary_filename}")
print("\nIntegrated yields:")
print(summary_df)

Data exported to: ionization_rates_450nm_8.0e+13Wcm2.csv
DataFrame shape: (1332, 10)

First few rows:
   time_SFA      rate_SFA  time_extended_1  rate_extended_1  time_extended_2  \
0   -166.00  4.258975e-34          -166.00     4.260377e-34          -166.00   
1   -165.75  2.615930e-32          -165.75     2.616798e-32          -165.75   
2   -165.50  6.372583e-31          -165.50     6.374721e-31          -165.50   
3   -165.25  8.556526e-30          -165.25     8.559432e-30          -165.25   
4   -165.00  7.568029e-29          -165.00     7.570637e-29          -165.00   

   rate_extended_2  time_extended_3  rate_extended_3  time_extended_4  \
0     4.260360e-34          -166.00     4.260362e-34          -166.00   
1     2.616781e-32          -165.75     2.616782e-32          -165.75   
2     6.374656e-31          -165.50     6.374658e-31          -165.50   
3     8.559309e-30          -165.25     8.559312e-30          -165.25   
4     7.570490e-29          -165.00     7.570493e-29

In [8]:
print(np.trapz(rate_SFA, time_recon))
print(np.real(np.trapz(rateExcited_1, time_recon_1)))
print(np.real(np.trapz(time_recon_2, rateExcited_2)))
print(np.real(np.trapz(time_recon_3, rateExcited_3)))
print(np.real(np.trapz(time_recon_4, rateExcited_4)))

0.00011346179704394354
0.00014147482586329002
-0.00010939851885141188
-0.00010049079038137944
-0.00010876240978240904


In [10]:
%autoreload
# data = plotter(params, time_recon, rate_SFA, rateExcited_1, rateExcited_2, rateExcited_3, useTex=False)

# data.matplot4()

fig = go.Figure()
fig.add_trace(go.Scatter(x=time_recon, y=rate_SFA, mode='lines', name='normal SFA'))
fig.add_trace(go.Scatter(x=time_recon_1, y=np.real(rateExcited_1), mode='lines', name='extended SFA_1'))
fig.add_trace(go.Scatter(x=time_recon_2, y=np.real(rateExcited_2), mode='lines', name='extended SFA_2'))
fig.add_trace(go.Scatter(x=time_recon_3, y=np.real(rateExcited_3), mode='lines', name='SFA_excited_3'))
fig.add_trace(go.Scatter(x=time_recon_4, y=np.real(rateExcited_4), mode='lines', name='SFA_excited_4'))
lam0 = params['lam0']
intensity = params['intensity']
fig.update_layout(
    title=f'Ionization Rate for {lam0} nm, {intensity:.2e} W/cm²',
    xaxis_title='Time (a.u.)',
    yaxis_title='Ionization Rate (a.u.)',
    legend=dict(x=0.01, y=0.99),
    template='plotly_white',
    xaxis_range=[-100, 100],
    width=800,
    height=600
)
fig.show()

# filename = f"plots/ionRate_{params['lam0']}_c0-is-1.html"
# if not os.path.exists(filename):
#     #fig.write_html(filename)
#     print(f"Plot saved as {filename}")
# else:
#     print(f"File {filename} already exists - skipping save")